In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2022-06-12 02:44:39--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M  46.8MB/s    in 1.8s    

2022-06-12 02:44:41 (46.8 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8

In [1]:
!python --version

Python 3.8.1


In [2]:
!pip install './snowflake_snowpark_python-0.7.0-py3-none-any.whl[pandas]'


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
#!pip install snowflake-connector-python

In [2]:

import sys, site

print("\n##### sys.path original #####")
for p in sys.path:
    print(f"'{p}'")
print()

site.main()

print("\n##### sys.path after site.main() #####")
for p in sys.path:
    print(f"'{p}'")
print()



##### sys.path original #####
'/content'
'/env/python'
'/usr/local/lib/python38.zip'
'/usr/local/lib/python3.8'
'/usr/local/lib/python3.8/lib-dynload'
''
'/usr/local/lib/python3.8/site-packages'
'/usr/local/lib/python3.8/site-packages/portpicker-1.3.1-py3.8.egg'


##### sys.path after site.main() #####
'/content'
'/env/python'
'/usr/local/lib/python38.zip'
'/usr/local/lib/python3.8'
'/usr/local/lib/python3.8/lib-dynload'
'/usr/local/lib/python3.8/site-packages'
'/usr/local/lib/python3.8/site-packages/portpicker-1.3.1-py3.8.egg'



In [3]:
!python --version

Python 3.8.1


In [4]:
!jupyter kernelspec list

Available kernels:
  py38       /root/.local/share/jupyter/kernels/py38
  ir         /usr/local/share/jupyter/kernels/ir
  python3    /usr/local/share/jupyter/kernels/python3


In [5]:
sys.version

'3.8.1 (default, Jan  8 2020, 22:29:32) \n[GCC 7.3.0]'

In [6]:
import os
import pandas as pd
import numpy as np

#import snowflake

from snowflake.snowpark import Session
from snowflake.snowpark.functions import col

In [7]:
connection_parameters = {
 "account": "nope.us-east-1",
 "user": "nope",
 "password": "nope",
 "role": "SYSADMIN",
 "warehouse": "ANALYTICS_WH",
 "database": "HURRICANE_ANALYSIS",
 "schema": "PUBLIC"
}

In [8]:
test_session = Session.builder.configs(connection_parameters).create()


In [9]:
df_disaster_decl_table = test_session.table("DISASTER_DECLARATIONS_PREPROC")
df_disaster_decl_table.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FEMADECLARATIONSTRING"  |"DISASTERNUMBER"  |"STATE"  |"DECLARATIONTYPE"  |"DECLARATIONDATE"         |"FYDECLARED"  |"INCIDENTTYPE"  |"DECLARATIONTITLE"        |"INCIDENTBEGINDATE"  |"INCIDENTENDDATE"  |"DISASTERCLOSEOUTDATE"  |"DESIGNATEDAREA"  |"FIPS_CODE"  |"DISASTER_DATE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|DR-7-MA                  |7                 |MA       |DR                 |1953-06-11T00:00:00.000Z  |1953          |Tornado         |TORNADO               

In [10]:
df_housing_assistance_table = test_session.table("FEMA_HOUSING_ASSISTANCE")
df_housing_assistance_table.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"DISASTERNUMBER"  |"STATE"  |"COUNTY"         |"CITY"      |"ZIPCODE"  |"VALIDREGISTRATIONS"  |"AVERAGEFEMAINSPECTEDDAMAGE"  |"TOTALINSPECTED"  |"TOTALDAMAGE"       |"NOFEMAINSPECTEDDAMAGE"  |"FEMAINSPECTEDDAMAGEBETWEEN1AND10000"  |"FEMAINSPECTEDDAMAGEBETWEEN10001AND20000"  |"FEMAINSPECTEDDAMAGEBETWEEN20001AND30000"  |"FEMAINSPECTEDDAMAGEGREATERTHAN30000"  |"APPROVEDFORFEMAASS

In [11]:
df_sql = test_session.sql("SELECT TOP 5 * from DISASTER_DECLARATIONS_PREPROC")
df_sql.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FEMADECLARATIONSTRING"  |"DISASTERNUMBER"  |"STATE"  |"DECLARATIONTYPE"  |"DECLARATIONDATE"         |"FYDECLARED"  |"INCIDENTTYPE"  |"DECLARATIONTITLE"        |"INCIDENTBEGINDATE"  |"INCIDENTENDDATE"  |"DISASTERCLOSEOUTDATE"  |"DESIGNATEDAREA"  |"FIPS_CODE"  |"DISASTER_DATE"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|DR-7-MA                  |7                 |MA       |DR                 |1953-06-11T00:00:00.000Z  |1953          |Tornado         |TORNADO               

In [12]:
df_joined_data = df_housing_assistance_table.join(df_disaster_decl_table, (df_housing_assistance_table.col("disasterNumber") == df_disaster_decl_table.col("disasterNumber")) & (df_housing_assistance_table.col("state") == df_disaster_decl_table.col("state")) & (df_housing_assistance_table.col("county") == df_disaster_decl_table.col("designatedArea")) )
#  
df_joined_data.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"l_kreu_DISASTERNUMBER"  |"l_kreu_STATE"  |"COUNTY"       

In [13]:
from snowflake.snowpark.functions import col, lit, sum as sum_, max as max_
from snowflake.snowpark.functions import month, year

df_tmp = df_joined_data.select((col("county"), col("incidentType"), col("totalDamage"), col("fips_code"), col("declarationTitle"), col("DISASTER_DATE"), month("DISASTER_DATE").as_("month"), year("DISASTER_DATE").as_("year")))
#df_joined_data.select((year("DISASTER_DATE")).alias("year"))




df_tmp.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------
|"COUNTY"         |"INCIDENTTYPE"   |"TOTALDAMAGE"       |"FIPS_CODE"  |"DECLARATIONTITLE"                     |"DISASTER_DATE"  |"MONTH"  |"YEAR"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------
|Walker (County)  |Severe Storm(s)  |120602.52           |48471        |SEVERE STORMS, TORNADOES AND FLOODING  |2002-11-05       |11       |2002    |
|Walker (County)  |Severe Storm(s)  |43803.6             |48471        |SEVERE STORMS, TORNADOES AND FLOODING  |2002-11-05       |11       |2002    |
|Jasper (County)  |Severe Storm(s)  |1703.64             |48241        |SEVERE STORMS, TORNADOES AND FLOODING  |2002-11-05       |11       |2002    |
|Jasper (County)  |Severe Storm(s)  |168300.7            |48241        |SEVERE STORMS, TORNADOES AND

In [14]:

df_grouped_sums = df_tmp.group_by(['county', 'fips_code', 'year', 'month', 'declarationTitle', 'incidentType']).agg(sum_("totalDamage"))
df_grouped_sums.show()

--------------------------------------------------------------------------------------------------------------------------------------------------
|"COUNTY"                        |"FIPS_CODE"  |"YEAR"  |"MONTH"  |"DECLARATIONTITLE"                     |"INCIDENTTYPE"   |"SUM(TOTALDAMAGE)"  |
--------------------------------------------------------------------------------------------------------------------------------------------------
|Walker (County)                 |48471        |2002    |11       |SEVERE STORMS, TORNADOES AND FLOODING  |Severe Storm(s)  |385689.48           |
|Montgomery (County)             |48339        |2002    |11       |SEVERE STORMS, TORNADOES AND FLOODING  |Severe Storm(s)  |1308098.97          |
|San Jacinto (County)            |48407        |2002    |11       |SEVERE STORMS, TORNADOES AND FLOODING  |Severe Storm(s)  |1292429.47          |
|Fort Bend (County)              |48157        |2002    |11       |SEVERE STORMS, TORNADOES AND FLOODING  |Severe Stor

In [15]:
#.filter(col("id") == 1)
df_hurricane_sums = df_grouped_sums.filter(col("incidentType") == "Hurricane")
df_hurricane_sums.show()

----------------------------------------------------------------------------------------------------------------------
|"COUNTY"               |"FIPS_CODE"  |"YEAR"  |"MONTH"  |"DECLARATIONTITLE"   |"INCIDENTTYPE"  |"SUM(TOTALDAMAGE)"  |
----------------------------------------------------------------------------------------------------------------------
|Matagorda (County)     |48321        |2003    |7        |HURRICANE CLAUDETTE  |Hurricane       |3005146.42          |
|Brazoria (County)      |48039        |2003    |7        |HURRICANE CLAUDETTE  |Hurricane       |1086684.0899999999  |
|Galveston (County)     |48167        |2003    |7        |HURRICANE CLAUDETTE  |Hurricane       |739854.5            |
|DeWitt (County)        |48123        |2003    |7        |HURRICANE CLAUDETTE  |Hurricane       |46700.92            |
|McMullen (County)      |48311        |2003    |7        |HURRICANE CLAUDETTE  |Hurricane       |30906.440000000002  |
|San Patricio (County)  |48409        |2003    |

In [16]:
test_session.close()